In [1]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import os
import openai
import pandas as pd
from neo4j.exceptions import CypherSyntaxError


In [2]:
curated_data = pd.read_csv('../data/benchmark_data/true_false_questions.csv').drop('Unnamed: 0', axis=1)


In [10]:
LLM_MODEL = 'gpt-4-32k'


In [11]:
def get_neo4j_cypher_rag_chain():
    load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
    username = os.environ.get('NEO4J_USER')
    password = os.environ.get('NEO4J_PSW')
    url = os.environ.get('NEO4J_URL')
    database = os.environ.get('NEO4J_DB')

    graph = Neo4jGraph(
        url=url, 
        username=username, 
        password=password,
        database = database
    )

    load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
    API_KEY = os.environ.get('API_KEY')
    API_VERSION = os.environ.get('API_VERSION')
    RESOURCE_ENDPOINT = os.environ.get('RESOURCE_ENDPOINT')
    openai.api_type = "azure"
    openai.api_key = API_KEY
    openai.api_base = RESOURCE_ENDPOINT
    openai.api_version = API_VERSION
    chat_deployment_id = LLM_MODEL
    chat_model_id = chat_deployment_id
    temperature = 0
    chat_model = ChatOpenAI(openai_api_key=API_KEY, 
                            engine=chat_deployment_id, 
                            temperature=temperature)
    chain = GraphCypherQAChain.from_llm(
        chat_model, 
        graph=graph, 
        verbose=True, 
        validate_cypher=True,
        return_intermediate_steps=True
    )
    return chain

In [12]:
%%time
neo4j_rag_chain = get_neo4j_cypher_rag_chain()


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


CPU times: user 52.7 ms, sys: 30.5 ms, total: 83.2 ms
Wall time: 21.4 s


In [14]:
question = curated_data.iloc[0]['text']
question

'enhanced S-cone syndrome is not a vitreoretinal degeneration'

In [15]:

neo4j_rag_answer = neo4j_rag_chain.run(query=question, return_final_only=True, verbose=False)
neo4j_rag_answer



> Entering new GraphCypherQAChain chain...


AttributeError: module 'openai' has no attribute 'error'